<a href="https://colab.research.google.com/github/brigitte-castaneda/PAE/blob/main/Scripts/Notebooks/Optimal_path_Demo_Fusagasuga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1. Confirmo las credenciales. 

In [3]:
from google.colab import auth
auth.authenticate_user()

# change these to try this notebook out
project = 'ph-jabri'
Dataset = 'WorldBank'
Tabla = 'directions'

import time
import os
 
os.environ['PROJECT'] = project
 

# BigQuery API
from google.cloud import bigquery 
import chardet
import pandas as pd
import pandas_gbq
import pprint
import time
import re
import numpy as np
bq_client = bigquery.Client(project=project)
# !pip install  pyproj
# !pip install progress

#Step 2. Se descarga la base desde el datalake en Bigquery de Google Cloud Platform

In [ ]:
sql = """
SELECT * EXCEPT(geomap) FROM `ph-jabri.WorldBank.direction_clean` where ETC like '%FUSAGA%' 
"""
df = bq_client.query(sql).to_dataframe() 

In [ ]:
df

,id_origen,id_destino,elevacion_origen,elevacion_destino,elevacion_ganada,distancia,elevacion_ganada_x_km,duracion,velocidad_km_h,distancia_destino_end_point,long_lat_origen,long_lat_destino,ETC
0,225290000914,0,1577.000000000,1608.000000000,31.000000000,3972.100000000,7.804435941,353.800000000,40.417071701,23.120226792,"-74.39413155,4.327306399","-74.3791805,4.3459142",FUSAGASUGA
1,225290001759,0,1812.000000000,1608.000000000,-204.000000000,11107.100000000,-18.366630354,815.700000000,49.019933871,23.120226792,"-74.333331,4.387887","-74.3791805,4.3459142",FUSAGASUGA
2,225290000469,0,1532.000000000,1608.000000000,76.000000000,12323.500000000,6.167079158,1197.600000000,37.044589141,23.120226792,"-74.384179,4.280602","-74.3791805,4.3459142",FUSAGASUGA
3,225290000850,0,1659.000000000,1608.000000000,-51.000000000,12557.900000000,-4.061188575,1040.200000000,43.461295971,23.120226792,"-74.358331,4.387886","-74.3791805,4.3459142",FUSAGASUGA
4,225290001643,0,1233.000000000,1608.000000000,375.000000000,11980.200000000,31.301647719,833.100000000,51.768959234,23.120226792,"-74.451492,4.295064","-74.3791805,4.3459142",FUSAGASUGA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3480,0,425290000662,1608.000000000,1544.000000000,-64.000000000,4843.400000000,-13.213858034,406.700000000,42.872485946,34.625962837,"-74.3791805,4.3459142","-74.399299759,4.327240719",FUSAGASUGA
3481,125290000103,425290000662,1716.000000000,1544.000000000,-172.000000000,4764.300000000,-36.101840774,470.200000000,36.476988547,34.625962837,"-74.36323115,4.338082518","-74.399299759,4.327240719",FUSAGASUGA
3482,125290000111,425290000662,1687.000000000,1544.000000000,-143.000000000,5764.800000000,-24.805717458,496.400000000,41.807574503,34.625962837,"-74.365787866,4.344406114","-74.399299759,4.327240719",FUSAGASUGA
3483,125290000146,425290000662,1740.000000000,1544.000000000,-196.000000000,4669.100000000,-41.978111413,434.500000000,38.685293583,34.625962837,"-74.364914724,4.329242742","-74.399299759,4.327240719",FUSAGASUGA


## Step 2.1. Se cambia el formato de datos, de objeto a numero.

In [ ]:
columnas = ['elevacion_origen','elevacion_destino', 'elevacion_ganada', 'distancia', 'elevacion_ganada_x_km',
            'duracion', 'velocidad_km_h', 'distancia_destino_end_point'  ]
lista = []
for i in columnas:
  df[i] = pd. to_numeric(df[i]) 
  var_interes =  i
  temp = df[['id_origen','id_destino', var_interes ]].pivot_table( values=var_interes, index=['id_origen'],
                    columns=['id_destino'], aggfunc=np.sum).to_numpy()
  lista.append(temp)        

# df.dtypes

In [ ]:
var_interes =  'distancia'
A = df[['id_origen','id_destino', var_interes ]].pivot_table( values=var_interes, index=['id_origen'], columns=['id_destino'], aggfunc=np.sum)

In [ ]:
A.index

Index(['0', '125290000049', '125290000103', '125290000111', '125290000120',
       '125290000146', '125290000171', '125290000189', '125290000201',
       '125290000219', '125290000251', '125290000570', '125290000626',
       '125290000839', '125290000863', '125290000936', '125290001118',
       '125290001355', '125290001835', '125290002234', '125290002246',
       '225290000248', '225290000272', '225290000281', '225290000299',
       '225290000311', '225290000337', '225290000345', '225290000353',
       '225290000370', '225290000388', '225290000396', '225290000418',
       '225290000426', '225290000451', '225290000469', '225290000477',
       '225290000485', '225290000493', '225290000507', '225290000515',
       '225290000540', '225290000604', '225290000680', '225290000736',
       '225290000795', '225290000850', '225290000914', '225290001074',
       '225290001198', '225290001309', '225290001317', '225290001473',
       '225290001643', '225290001651', '225290001660', '225290001741',
 

# Step 3. Se crean las matrices que van a ser usadas para el calculo. 

In [ ]:
distancia_matriz = lista[3]
distancia_matriz

array([[    0. ,  3429.1,  2453.5, ...,  2446. , 10807.5,  4843.4],
       [ 3557.2,     0. ,   395. , ...,  1533.5,  9432.5,  4713.3],
       [ 2560.5,   431. ,     0. , ...,   784.4,  9055.1,  4764.3],
       ...,
       [ 2401.6,  1459.4,  1087. , ...,     0. ,  8428.7,  5891.8],
       [11107.1,  9780.7,  9408.2, ...,  8421.4,     0. , 14597.3],
       [ 4843.4,  4691.8,  4735.8, ...,  5898.5, 13548.8,     0. ]])

#Step 4. Se establece la optimizacion según distancia 

In [ ]:

class descarga_dist_btw_points:
  #conductor
    def __init__(self,distancia_matriz = None, num_vehicles= None ,  Centro_acopio=0, vehicle_maximum_dist  =None  ):  
      self.distancia_matriz = distancia_matriz
      self.num_vehicles = num_vehicles
      self.depot = Centro_acopio
      self.vehicle_maximum_dist = vehicle_maximum_dist
       
    def import_or_install(package):
      try:
        __import__(package)
      except ImportError:
        pip.main(['install', package])  

    import_or_install('ortools')

    from ortools.constraint_solver import routing_enums_pb2
    from ortools.constraint_solver import pywrapcp


    def create_data_model(self):
        """Stores the data for the problem."""
        data = {}
        data['distance_matrix'] =  self.distancia_matriz
        data['num_vehicles'] = self.num_vehicles
        data['depot'] = self.depot
        return data


    def print_solution(self):
        """Prints solution on console."""
        print(f'Objective: {self.solution.ObjectiveValue()}')
        max_route_distance = 0
        for vehicle_id in range( self.num_vehicles ):
            index = self.routing.Start(vehicle_id)
            plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
            route_distance = 0
            while not self.routing.IsEnd(index):
                plan_output += ' {} -> '.format(self.manager.IndexToNode(index))
                previous_index = index
                index = self.solution.Value(self.routing.NextVar(index))
                route_distance += self.routing.GetArcCostForVehicle(
                    previous_index, index, vehicle_id)
            plan_output += '{}\n'.format(self.manager.IndexToNode(index))
            plan_output += 'Distance of the route: {}m\n'.format(route_distance)
            print(plan_output)
            max_route_distance = max(route_distance, max_route_distance)
        print('Maximum of the route distances: {}m'.format(max_route_distance))



    def main(self):
        """Entry point of the program."""
        # Instantiate the data problem.
        self.Data = self.create_data_model()  

        # Create the routing index manager.
        self.manager = pywrapcp.RoutingIndexManager(len(self.Data['distance_matrix']),
                                              self.Data['num_vehicles'], self.Data['depot'])

        # Create Routing Model.
        self.routing = pywrapcp.RoutingModel(self.manager)


        # Create and register a transit callback.
        def distance_callback(self, from_index, to_index):
            """Returns the distance between the two nodes."""
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = self.manager.IndexToNode(from_index)
            to_node = self.manager.IndexToNode(to_index)
            return self.data['distance_matrix'][from_node][to_node]

        self.transit_callback_index = self.routing.RegisterTransitCallback(distance_callback)


        # Create and register a transit callback.
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        # self.from_node = self.manager.IndexToNode(from_index)
        # self.to_node = self.manager.IndexToNode(to_index)
 
        # self.distance_callback = self.Data['distance_matrix'][self.from_node][self.to_node]
        # self.transit_callback_index = self.routing.RegisterTransitCallback(self.distance_callback)

        # Define cost of each arc.
        self.routing.SetArcCostEvaluatorOfAllVehicles(self.transit_callback_index)

        # Add Distance constraint.
        self.dimension_name = 'Distance'
        self.routing.AddDimension(
            self.transit_callback_index,
            0,  # no slack
            self.vehicle_maximum_dist,  # vehicle maximum travel distance
            True,  # start cumul to zero
            self.dimension_name)
        
        self.distance_dimension = self.routing.GetDimensionOrDie(self.dimension_name)
        self.distance_dimension.SetGlobalSpanCostCoefficient(100)

        # Setting first solution heuristic.
        self.search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        self.search_parameters.first_solution_strategy = ( routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

        # Solve the problem.
        self.solution = self.routing.SolveWithParameters(self.search_parameters)

        # Print solution on console.
        if self.solution:
            self.print_solution()
        else:
            print('No solution found !')


    # if __name__ == '__main__':
    #     self.main()

In [ ]:
A = descarga_dist_btw_points( distancia_matriz = distancia_matriz, num_vehicles= 4 , vehicle_maximum_dist  =60000  ).main()

Objective: 4145801
Route for vehicle 0:
 0 ->  24 ->  41 ->  46 ->  57 ->  45 ->  40 ->  36 ->  16 ->  9 ->  15 ->  52 ->  8 ->  50 ->  18 ->  3 ->  56 ->  25 ->  26 ->  49 ->  30 ->  44 ->  28 -> 0
Distance of the route: 39950m

Route for vehicle 1:
 0 ->  29 ->  20 ->  58 ->  19 ->  37 ->  32 ->  35 ->  54 ->  34 ->  17 ->  22 ->  42 ->  39 ->  55 ->  12 ->  5 ->  51 ->  43 ->  1 ->  10 ->  33 ->  13 ->  6 ->  2 ->  14 ->  11 ->  27 -> 0
Distance of the route: 38346m

Route for vehicle 2:
 0 ->  4 ->  48 ->  21 ->  38 ->  31 -> 0
Distance of the route: 39953m

Route for vehicle 3:
 0 ->  23 ->  53 ->  7 ->  47 -> 0
Distance of the route: 32252m

Maximum of the route distances: 39953m


In [6]:
# %%writefile Simple Vehicles Routing.py
"""Simple Vehicles Routing Problem (VRP) - 55 sedes educativas FUSAGASUGA.

   This is a sample using the routing library python wrapper to solve a VRP
   problem.
   A description of the problem can be found here:
   http://en.wikipedia.org/wiki/Vehicle_routing_problem.

   Distances are in meters.
"""

     
import pip
import os

def import_or_install(package):
    try:
        __import__(package)
    except ImportError:
        pip.main(['install', package])  


import_or_install('ortools')

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model(distancia_matriz):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] =  distancia_matriz
    data['num_vehicles'] = 4
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))



def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model(distancia_matriz)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        50000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print('No solution found !')

if __name__ == '__main__':
    main()



Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

Collecting ortools

Downloading ortools-9.5.2237-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.3 MB)

Output()

...

Uninstalling protobuf-3.19.6:

Successfully uninstalled protobuf-3.19.6

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.
tensorflow-metadata 1.12.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.12 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.


Successfully installed ortools-9.5.2237 protobuf-4.21.12

NameError: ignored

In [ ]:
lista[4]

id_destino,0,125290000049,125290000103,125290000111,125290000120,125290000146,125290000171,125290000189,125290000201,125290000219,...,225290001074,225290001198,225290001309,225290001317,225290001473,225290001643,225290001651,225290001741,225290001759,425290000662
id_origen,,,,,,,,,,,,,,,,,,,,,
0,0.000000,35.432721,27.456015,27.693778,30.944933,33.192086,36.437788,39.794203,35.361189,37.761841,...,31.351641,31.197558,41.145658,36.880678,36.485067,45.447482,32.229337,29.342219,48.259737,42.872486
125290000049,36.012148,0.000000,15.000000,24.596415,23.995393,25.776347,32.073198,35.467475,34.566234,38.578553,...,29.637473,27.092354,38.098537,29.233082,36.414347,43.177251,25.955674,26.566891,50.614100,37.365955
125290000103,31.385087,15.005803,0.000000,15.316981,19.904549,25.154305,20.337870,34.776013,31.195503,35.444879,...,27.766837,17.397626,35.630236,26.969715,32.851901,42.740511,25.863704,15.380392,50.166759,36.476989
125290000111,31.873102,29.730753,17.157447,0.000000,22.829345,30.540834,29.546767,39.363069,17.221341,27.527273,...,29.262180,15.000000,23.351663,32.347020,24.460212,44.813161,26.958413,15.007848,48.231729,41.807575
125290000120,32.587488,23.913962,18.785996,18.265792,0.000000,26.855226,20.300885,39.038076,27.778815,26.460254,...,21.099108,17.972675,33.273684,28.577872,29.164931,44.394265,26.124321,17.821078,48.889551,41.173377
125290000146,32.818186,25.776347,25.154305,26.359118,26.945170,0.000000,32.662002,36.489287,34.214072,37.110581,...,30.178298,28.745795,36.975775,18.198738,35.511794,43.782353,25.092840,28.303448,48.896453,38.685294
125290000171,36.968553,33.220000,24.276504,22.184100,17.568664,33.219082,0.000000,37.158663,35.841330,42.178805,...,29.146698,24.421020,40.466364,38.352565,38.983646,43.702360,26.875104,23.822993,52.762687,39.187650
125290000189,39.794203,38.471083,37.088404,36.864286,35.158380,37.698989,40.608442,0.000000,37.369139,38.645301,...,34.320420,39.225209,36.154920,39.622613,38.010790,40.751221,28.605792,38.247517,45.866643,20.791274
125290000201,37.688058,35.971340,32.562225,14.999419,25.571577,35.295861,37.865009,41.372415,0.000000,36.765875,...,32.149327,16.833933,31.103337,38.037789,31.807811,45.502081,27.894371,16.956856,52.493509,43.768454


In [ ]:
df = pd.DataFrame(lista[4])

Exportar velocidades a csv

In [ ]:
df.to_csv('matrix_tiempo_fusa.csv')

Exportar distancias a csv

In [ ]:
lista[1]

In [ ]:
df = pd.DataFrame(lista[1])
df.to_csv('matrix_dis_fusa.csv')

# Ejercicio:
**Restricciones de capacidad + VRPC 55 sedes educativas de fusagasuga**


In [7]:
"""Capacited Vehicles Routing Problem (CVRP)."""
    
import pip
import os

def import_or_install(package):
    try:
        __import__(package)
    except ImportError:
        pip.main(['install', package])  


import_or_install('ortools')

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
[0,3429.1,2453.5,2608.6,2875.3,3867.8,2946.4,5491.6,3371.1,3463.6,3611.8,2945.7,4035.4,3203.8,2923.4,3231.3,3884.6,4223.7,3889.1,4589.4,4493.4,3992.2,4740,2574.5,2565,3004.6,2492.2,4515.6,2467.1,4031.8,7578.5,3143.2,9071.5,12323.5,7156.4,7578.5,4019.3,8432.5,10771.9,4010.2,3701.6,2478.9,11578.1,12258.2,3972.1,4775.9,2555.6,3987.7,3865.3,3264.4,13985.2,12426.2,2446,10807.5,4843.4],
[3557.2,0,395,1943.8,1041.8,890,618.3,5361.5,1996.2,2088.6,183.6,662.3,1440,404.2,639.9,1856.3,2509.7,1306,2514.2,4459.3,4363.3,1141.6,7762.6,1514.9,1505.5,721.2,1570.7,4385.4,1545.6,1114.1,7448.3,286,8941.3,12193.3,7513.4,7448.3,1168.7,7057.5,11256.4,1159.6,484.3,1557.5,10203.1,10883.3,3841.9,3400.9,1496.1,2603.4,648,1889.5,13855.1,9225.8,1533.5,9432.5,4713.3],
[2560.5,431,0,947.1,790.1,1266.1,381.9,5412.5,1618.7,1711.2,613.7,751.4,1816.1,362.7,729.1,1478.9,2132.2,1682.1,2136.7,4510.3,4414.3,1517.7,6765.9,873.8,864.4,810.4,830.6,4436.4,805.5,1490.2,7499.3,302.1,8992.3,12244.3,7135.9,7499.3,1544.8,6680.1,10879,1535.7,860.4,817.4,9825.7,10505.8,3892.9,3023.5,854.9,2225.9,1024.1,1512.1,13906.1,9601.9,784.4,9055.1,4764.3],
[2274.5,2006,1030.4,0,1452.2,2684.2,1523.3,6412.9,663.5,1286.9,2188.7,1522.6,3234.1,1780.7,1500.3,1054.6,1708,3100.2,743.6,5510.8,5414.8,2935.7,6479.9,372.5,381,1581.5,396.8,5436.9,371.6,2908.3,8499.8,1720.1,9992.8,13244.8,6711.7,8499.8,2962.8,6255.9,10454.8,2953.8,2278.5,383.5,9401.5,10081.6,4893.4,3393.6,390,1014.5,2442.2,1087.8,14906.6,11020,350.6,8630.8,5764.8],
[2850.5,1056.2,819.8,1237,0,1734.4,573.5,6988.9,1289.4,1039.3,1238.9,1639.2,2284.3,831,1616.8,1149.6,1557.5,2150.4,1807.4,6086.7,5990.7,1986,7055.8,808.2,798.7,1698.1,864,6012.9,838.8,1958.5,9075.8,770.4,10568.8,13820.8,6464,9075.8,2013,6008.2,10207.1,2004,1328.7,850.7,9153.8,9834,5469.4,1642.8,789.3,1896.6,1492.4,1182.8,15482.5,10070.2,826.7,8383.2,6340.7],
[3829.7,890,1266.1,2622,1720,0,1296.5,5317.3,2674.4,2766.8,1105.8,1249.9,554.9,1082.4,1227.5,2534.5,3187.9,744.2,3192.4,4415.1,4319.2,512.7,8035,2193.1,2183.7,1308.8,2248.9,4341.3,2223.8,552.3,7404.2,964.2,8897.2,12149.2,8191.6,7404.2,539.8,7735.7,11934.6,530.7,336.4,2235.7,10881.3,11561.5,3797.8,4079.1,2174.3,3281.6,320.5,2567.7,13810.9,8664,2211.6,10110.7,4669.1],
[2939,830.5,594.1,1325.5,423.6,1508.7,0,6112.6,1377.9,1470.4,1013.2,1413.5,2058.6,605.2,1391.1,1238.1,1891.4,1924.7,1895.9,5210.5,5114.5,1760.2,7144.3,896.7,887.2,1472.4,952.5,5136.6,927.3,1732.8,8199.5,544.7,9692.5,12944.5,6895.1,8199.5,1787.3,6439.3,10638.2,1778.3,1103,939.2,9584.9,10265.1,4593.1,2782.7,877.8,1985.1,1266.7,1271.3,14606.3,9844.5,915.2,8814.3,5464.5],
[5491.6,5340,5384,6709.3,5932,5281,5879.2,0,6760.7,6853.1,5362,5072.7,5448.6,5665.2,5050.4,6620.8,7274.1,5637,7278.6,902.2,1015,5405.4,5973.1,6675.1,6665.7,5131.6,6592.9,1037.1,6567.7,5445.1,4755.9,5546.9,6248.9,9500.9,9843.2,4755.9,5432.5,11822,13458.8,5423.5,5453,6579.6,14967.6,15647.8,1543.2,8165.4,6656.3,6674.6,5256.6,6654,9412.4,9603.6,6546.7,14197,979.5],
[3585.6,2259.2,1886.8,430.4,1369.5,2937.4,1776.5,7724,0,945.7,2441.9,2379,3487.3,2033.9,2356.7,713.4,1366.8,3353.4,661.6,6821.9,6725.9,3188.9,7791,528.2,536.7,2437.9,552.5,6748,527.4,3161.5,9810.9,1973.4,11303.9,14555.9,6370.4,9810.9,3216,5914.6,10113.5,3207,2531.7,539.3,9060.2,9740.4,6204.5,3052.4,545.7,750.8,2695.4,746.6,16217.7,11273.2,515.3,8289.6,7075.9],
[3763.2,2436.8,2064.3,1307.1,1114.9,3115,1954,7901.6,877.6,0,2619.5,2556.6,3664.9,2211.5,2534.2,262.2,518.5,3531,1395.6,6999.4,6903.5,3366.5,7968.6,1090.5,1099,2615.5,1114.8,6925.6,1089.6,3339.1,9988.5,2150.9,11481.5,14733.5,5425,9988.5,3393.6,4969.2,9168.1,3384.5,2709.3,1101.5,8114.8,8795,6382.1,2366.8,1108,1484.8,2873,425.9,16395.2,11450.8,1077.5,7344.2,7253.4],
[2772,378.7,211.4,1158.5,948,1213.8,524.4,5817.8,1902.4,1994.8,0,1118.6,1763.7,310.4,1096.2,1762.5,2415.9,1629.8,2420.3,4915.6,4819.6,1465.4,6977.3,1421.1,1411.7,1177.5,1476.9,4841.8,1451.8,1437.9,7904.7,249.8,9397.7,12649.6,7419.5,7904.7,1492.4,6963.7,11162.6,1483.4,808.1,1463.7,10109.3,10789.5,4298.3,3307.1,1402.2,2509.6,971.8,1795.7,14311.4,9549.6,1439.6,9338.7,5169.6],
[2629.8,1285.5,311.3,1016.3,859.3,1638.9,802.8,5466.7,1688,1780.4,1095,0,2188.8,1060.3,783.4,1548.1,2201.5,2054.9,2206,4564.5,4468.6,1890.4,6835.1,943,933.6,58.9,899.9,4490.7,874.7,1863,7553.6,999.7,9046.6,12298.6,7205.1,7553.6,1917.5,6749.3,10948.2,1908.5,1233.2,886.6,9894.9,10575.1,3947.2,3092.7,924.2,2295.2,1396.9,1581.3,13960.3,9974.7,853.7,9124.3,4818.5],
[3986.2,1428.8,1804.9,3160.8,2258.8,554.9,1835.3,5473.9,3213.2,3305.6,1644.7,1788.7,0,1621.2,1766.3,3073.3,3726.7,554.2,3731.2,4571.7,4475.7,486.8,8191.6,2731.9,2722.5,1847.6,2787.8,4497.9,2762.6,381,7560.8,1503,6679.1,12305.7,8730.4,7560.8,513.9,8274.6,12473.4,504.9,1055.7,2774.5,11420.2,12100.3,3954.4,4618,2713.1,3820.4,859.3,3106.5,13967.5,8303.6,2750.5,10649.5,4825.7],
[3332,404.2,362.7,1718.5,816.6,1082.4,393,5686.4,1770.9,1863.4,586.9,987.2,1632.3,0,964.8,1631.1,2284.4,1498.4,2288.9,4784.2,4688.2,1333.9,7537.3,1289.7,1280.2,1046.1,1345.5,4710.3,1320.3,1306.5,7773.2,118.4,9266.2,12518.2,7288.1,7773.2,1361,6832.3,11031.2,1352,676.7,1332.2,9977.9,10658.1,4166.8,3175.7,1270.8,2378.2,840.4,1664.3,14180,9418.2,1308.2,9207.3,5038.2],
[2652.1,1307.9,333.6,1038.7,881.7,1661.3,825.1,5489.1,1710.3,1802.8,1117.3,22.4,2211.2,1082.6,0,1570.5,2223.8,2077.3,2228.3,4586.9,4490.9,1912.8,6857.5,965.4,956,81.3,922.2,4513.1,897.1,1885.4,7576,1022,9069,12320.9,7227.5,7576,1939.9,6771.7,10970.6,1930.8,1255.6,909,9917.3,10597.4,3969.6,3115.1,946.5,2317.5,1419.3,1603.6,13982.7,9997,876,9146.7,4840.9],
[3530.9,2204.5,1832,1074.8,1314.8,2882.7,1721.7,7669.3,645.3,262.2,2387.2,2324.3,3432.6,1979.2,2301.9,0,780.5,3298.7,1163.3,6767.1,6671.2,3134.2,7736.3,858.2,866.7,2383.2,882.5,6693.3,857.3,3106.8,9756.2,1918.6,11249.2,14501.2,5687,9756.2,3161.3,5231.2,9430,3152.2,2477,869.2,8376.8,9056.9,6149.8,2368.9,875.7,1252.5,2640.7,193.6,16162.9,11218.5,845.2,7606.1,7021.1],
[4281.4,2955,2582.6,1825.3,1633.1,3633.2,2472.3,8419.8,1395.9,518.5,3137.7,3074.8,4183.1,2729.7,3052.5,780.5,0,4049.2,1913.8,7517.7,7421.7,3884.7,8486.8,1608.7,1617.2,3133.7,1633,7443.8,1607.9,3857.3,10506.7,2669.1,11999.7,15251.7,5066.5,10506.7,3911.8,4610.7,8809.6,3902.8,3227.5,1619.7,7756.3,8436.5,6900.3,2885,1626.2,2003.1,3391.2,944.1,16913.4,11969,1595.7,6985.7,7771.7],
[4185.7,1304.6,1680.7,3036.5,2134.6,744.2,1711,5673.3,3088.9,3181.4,1520.4,1664.4,554.2,1496.9,1642,2949.1,3602.4,0,3606.9,4771.1,4675.1,253.2,8391,2607.7,2598.2,1723.3,2663.5,4697.3,2638.3,225.7,7760.2,1378.7,6409.8,12505.2,8606.1,7760.2,280.2,8150.3,12349.2,271.2,1255.2,2650.2,11295.9,11976,4153.8,4493.7,2588.8,3696.1,1058.8,2982.3,14166.9,8034.2,2626.2,10525.3,5025.1],
[2915.4,2709.1,2336.7,674,1819.4,3387.3,2226.4,7053.8,772.1,1395.6,2891.8,2828.9,3937.2,2483.8,2806.5,1163.3,1816.7,3803.3,0,6151.6,6055.7,3638.8,7120.8,978.1,986.6,2887.8,1002.4,6077.8,977.3,3611.4,9140.7,2423.2,10633.7,13885.7,6820.3,9140.7,3665.9,6364.5,10563.4,3656.9,2981.6,989.2,9510.1,10190.3,5534.3,3502.2,995.6,643.7,3145.3,1196.5,15547.4,11723.1,965.1,8739.5,6405.6],
[4589.4,4437.8,4481.8,5807.1,5029.8,4378.8,4977,902.2,5858.5,5950.9,4459.8,4170.5,4546.5,4763,4148.2,5718.6,6372,4734.8,6376.4,0,1406.3,4503.2,8794.8,5772.9,5763.5,4229.4,5690.7,1428.5,5665.5,4542.9,3853.7,4644.7,5346.7,8598.7,10392.6,3853.7,4530.3,10919.8,14008.2,4521.3,4550.8,5677.4,14065.4,14745.6,641,7263.2,5754.1,7224,4354.4,5751.8,9395.8,8701.4,5644.5,13294.8,254],
[4493.4,4341.8,4385.8,5711.2,4933.8,4282.8,4881.1,1015,5762.5,5854.9,4363.8,4074.5,4450.5,4667,4052.2,5622.6,6276,4638.8,6280.5,1406.3,0,4407.3,6336.6,5677,5667.5,4133.4,5594.7,22.1,5569.6,4446.9,4395.4,4548.8,5888.4,9140.4,10296.6,4395.4,4434.3,10823.8,13912.2,4425.3,4454.9,5581.5,13969.4,14649.6,768.4,7167.2,5658.1,7128,4258.5,5655.8,10802.1,9243.1,5548.5,13198.8,1036.2],
[3954.1,1140.1,1516.2,2872.1,1970.1,512.7,1546.5,5441.8,2924.5,3016.9,1355.9,1499.9,486.8,1332.5,1477.6,2784.6,3438,253.2,3442.4,4539.6,4443.6,0,8159.5,2443.2,2433.8,1558.8,2499,4465.8,2473.9,61.3,7528.7,1214.2,6548.5,12273.6,8441.6,7528.7,27.1,7985.8,12184.7,18,1023.6,2485.8,11131.4,11811.6,3922.3,4329.2,2424.4,3531.7,827.2,2817.8,13935.4,8172.9,2461.7,10360.8,4793.6],
[4792.7,7687.1,6711.5,6866.6,7133.3,8125.8,7204.3,9749.6,7629.1,7721.6,7869.8,7203.7,8293.4,7461.8,7181.4,7489.3,8142.6,8481.7,7391.1,8847.4,8751.4,8250.2,0,6832.4,6823,7262.6,6750.2,8773.6,6725,8289.8,11836.5,7401.2,13329.5,16581.5,9916,11836.5,8277.3,12690.5,8108.6,8268.2,7959.6,6736.9,15836.1,10888.5,8230.1,9033.9,6813.6,6747.3,8123.3,7522.4,13958.5,16684.2,6704,15065.5,9101.4],
[2414.6,1472.4,1100,343.3,918.6,2150.6,989.7,6553,472,1066.2,1655.1,1592.2,2700.5,1247.2,1569.9,834,1487.3,2566.6,964,5650.8,5554.8,2402.1,6619.9,0,9.5,1651.1,59.2,5576.9,34,2374.7,8639.9,1186.6,10132.9,13384.8,6491,8639.9,2429.2,6035.2,10234.1,2420.2,1744.9,45.9,9180.8,9860.9,5033.4,2819.2,19,1053.3,1908.6,867.1,15046.6,10486.4,21.9,8410.1,5904.8],
[2423,1480.9,1108.5,351.8,927.1,2159.1,998.2,6561.5,462.6,1056.8,1663.6,1600.7,2709,1255.6,1578.4,824.5,1477.9,2575.1,972.5,5659.3,5563.3,2410.6,6628.4,9.5,0,1659.6,67.7,5585.4,42.5,2383.2,8648.3,1195,10141.3,13393.3,6481.5,8648.3,2437.7,6025.7,10224.6,2428.7,1753.4,54.4,9171.3,9851.5,5041.9,2827.7,9.5,1061.8,1917.1,857.7,15055.1,10494.9,30.4,8400.7,5913.3],
[2588.6,1244.4,270.2,975.2,818.2,1597.8,761.6,5425.6,1646.9,1739.3,1053.8,764.6,2147.7,1019.1,742.2,1507,2160.3,2013.8,2164.8,4523.4,4427.4,1849.3,6794,901.9,892.5,0,858.8,4449.6,833.6,1821.9,7512.5,958.5,9005.5,12257.5,7164,7512.5,1876.4,6708.2,10907.1,1867.4,1192.1,845.5,9853.8,10534,3906.1,3051.6,883.1,2254.1,1355.8,1540.2,13919.2,9933.6,812.6,9083.2,4777.4],
[2355.4,1413.3,1040.8,446.7,859.5,2091.5,930.5,6493.8,607.5,1447.7,1595.9,1533.1,2641.4,1188,1510.7,1215.4,1868.8,2507.4,1067.4,5591.6,5495.7,2343,6560.8,302.6,294.6,1592,0,5517.8,301.8,2315.6,8580.7,1127.4,10073.7,13325.7,6872.4,8580.7,2370.1,6416.6,10615.5,2361,1685.8,313.7,9562.2,10242.4,4974.3,2760,285.2,1156.7,1849.5,1248.6,14987.4,10427.2,280.8,8791.6,5845.6],
[4515.6,4363.9,4407.9,5733.3,4956,4305,4903.2,1037.1,5784.6,5877.1,4385.9,4096.7,4472.6,4689.2,4074.3,5644.8,6298.1,4660.9,6302.6,1428.5,22.1,4429.4,6358.8,5699.1,5689.7,4155.6,5616.9,0,5591.7,4469,4417.5,4570.9,5910.5,9162.5,10318.8,4417.5,4456.5,10846,13934.3,4447.4,4477,5603.6,13991.6,14671.8,790.5,7189.4,5680.3,7150.1,4280.6,5678,10824.3,9265.3,5570.7,13221,1058.4],
[2380.6,1438.4,1066,471.9,884.6,2116.6,955.7,6519,632.7,1472.9,1621.1,1558.2,2666.5,1213.2,1535.9,1240.6,1893.9,2532.6,1092.6,5616.8,5520.8,2368.2,6585.9,327.8,319.8,1617.1,25.2,5543,0,2340.7,8605.9,1152.6,10098.9,13350.8,6897.6,8605.9,2395.2,6441.8,10640.7,2386.2,1710.9,11.9,9587.4,10267.6,4999.5,2785.2,310.3,1181.8,1874.6,1273.8,15012.6,10452.4,305.9,8816.8,5870.8],
[3993.8,1112.7,1488.8,2844.6,1942.7,552.3,1519.1,5481.4,2897,2989.5,1328.5,1472.5,381,1305,1450.1,2757.2,3410.5,225.7,3415,4579.2,4483.2,61.3,8199.1,2415.8,2406.3,1531.4,2471.6,4505.4,2446.4,0,7568.3,1186.8,6487.3,12313.3,8414.2,7568.3,88.3,7958.4,12157.3,79.3,1063.3,2458.3,11104,11784.1,3961.9,4301.8,2396.9,3504.2,866.9,2790.4,13975,8111.7,2434.3,10333.4,4833.2],
[7578.5,7426.8,7470.8,8796.2,8018.9,7367.9,7966.1,4755.9,8847.5,8940,7448.8,7159.6,7535.5,7752.1,7137.2,8707.7,9361,7723.8,9365.5,3853.7,4395.4,7492.3,11783.8,8762,8752.6,7218.5,8679.8,4417.5,8654.6,7531.9,0,7633.8,4623.8,5987.4,13381.7,0,7519.4,13908.9,16997.2,7510.3,7539.9,8666.5,17054.5,17734.7,3630.1,10252.3,8743.2,10213,7343.5,8740.9,13249.5,8178.6,8633.6,16283.9,4107.7],
[3271.4,286,302.1,1657.9,756,964.2,332.4,5568.1,1710.4,1802.8,468.6,868.9,1514.1,118.4,846.6,1570.5,2223.8,1380.2,2228.3,4665.9,4569.9,1215.7,7476.7,1229.1,1219.6,927.8,1284.9,4592.1,1259.8,1188.3,7655,0,9148,12400,7227.5,7655,1242.8,6771.7,10970.6,1233.7,558.5,1271.6,9917.3,10597.5,4048.6,3115.1,1210.2,2317.6,722.2,1603.7,14061.7,9299.9,1247.6,9146.7,4919.9],
[9071.5,8919.8,8963.8,10289.2,9511.9,8860.9,9459.1,6248.9,10340.5,10433,8941.8,8652.6,6679.1,9245.1,8630.2,10200.7,10854,6409.8,10858.5,5346.7,5888.4,8985.3,13276.8,10255,10245.6,8711.5,10172.8,5910.5,10147.6,6487.3,4623.8,9126.8,0,5250,14874.7,4623.8,9012.4,15401.9,18490.2,9003.3,9032.9,10159.5,18547.5,19227.7,5123.1,11745.3,10236.2,11706,8836.5,10233.9,14742.5,3513.7,10126.6,17776.9,5600.7],
[12323.5,12171.8,12215.8,13541.2,12763.9,12112.8,12711.1,9500.9,13592.5,13685,12193.8,11904.6,12280.5,12497,11882.2,13452.7,14106,12468.8,14110.5,8598.7,9140.4,12237.3,16528.8,13507,13497.6,11963.5,13424.7,9162.5,13399.6,12276.9,5987.4,12378.8,5250,0,18126.7,5987.4,12264.4,18653.9,21742.2,12255.3,12284.9,13411.5,21799.5,22479.6,8375.1,14997.3,13488.1,14958,12088.5,13485.8,17994.5,5396.2,13378.5,21028.9,8852.7],
[9187.9,7861.5,7489.1,6731.8,6539.6,8539.7,7378.8,13326.3,6302.4,5425,8044.2,7981.3,9089.6,7636.2,7959,5687,5066.5,8955.7,6820.3,12424.2,12328.2,8791.2,10862.3,6515.2,6523.7,8040.2,6539.5,12350.3,6514.4,8763.8,15413.2,7575.6,16906.2,20158.2,0,15413.2,8818.3,4402.3,8601.2,8809.3,8134,6526.2,7547.9,8228.1,11806.8,7791.5,6532.7,6909.6,8297.7,5850.6,21820,16875.5,6502.2,6777.3,12678.2],
[7578.5,7426.8,7470.8,8796.2,8018.9,7367.9,7966.1,4755.9,8847.5,8940,7448.8,7159.6,7535.5,7752.1,7137.2,8707.7,9361,7723.8,9365.5,3853.7,4395.4,7492.3,11783.8,8762,8752.6,7218.5,8679.8,4417.5,8654.6,7531.9,0,7633.8,4623.8,5987.4,13381.7,0,7519.4,13908.9,16997.2,7510.3,7539.9,8666.5,17054.5,17734.7,3630.1,10252.3,8743.2,10213,7343.5,8740.9,13249.5,8178.6,8633.6,16283.9,4107.7],
[3981.2,1167.2,1543.3,2899.1,1997.2,539.8,1573.6,5468.9,2951.6,3044,1383,1527,513.9,1359.6,1504.7,2811.7,3465,280.2,3469.5,4566.7,4470.7,27.1,8186.6,2470.3,2460.9,1585.9,2526.1,4492.8,2501,88.3,7555.7,1241.3,6529.6,12300.7,8468.7,7555.7,0,8012.9,12211.8,9.1,1050.7,2512.8,11158.5,11838.7,3949.3,4356.3,2451.4,3558.8,854.3,2844.9,13962.5,8154,2488.8,10387.9,4820.7],
[8732.1,7405.7,7033.3,6276,6083.8,8083.9,6922.9,12870.5,5846.6,4969.2,7588.4,7525.5,8633.8,7180.4,7503.1,5231.2,4610.7,8499.9,6364.5,11968.3,11872.4,8335.4,12937.5,6059.4,6067.9,7584.4,6083.7,11894.5,6058.6,8308,14957.4,7119.8,16450.4,19702.4,4402.3,14957.4,8362.5,0,4931.9,8353.5,7678.2,6070.4,3878.6,4558.8,11351,7335.7,6076.9,6453.8,7841.9,5394.8,21364.1,16419.7,6046.4,3108,12222.3],
[12931,11604.6,11232.2,10474.9,10282.7,12282.8,11121.8,17069.4,10045.5,9168.1,11787.3,11724.4,12832.7,11379.3,11702,9430,8809.6,12698.8,10563.4,16167.2,16071.3,12534.3,8737,10258.3,10266.8,11783.3,10282.6,16093.4,10257.5,12506.9,19156.3,11318.7,20649.3,23901.3,8601.2,19156.3,12561.4,4931.9,0,12552.3,11877.1,10269.3,4673.1,2781.4,15549.9,11534.6,10275.8,10652.7,12040.8,9593.7,25563,20618.6,10245.3,3902.5,16421.2],
[3972.1,1158.1,1534.2,2890.1,1988.1,530.7,1564.6,5459.8,2942.5,3034.9,1373.9,1518,504.9,1350.5,1495.6,2802.6,3456,271.2,3460.5,4557.6,4461.6,18,8177.5,2461.2,2451.8,1576.9,2517,4483.8,2491.9,79.3,7546.7,1232.3,6538.7,12291.7,8459.7,7546.7,9.1,8003.8,12202.7,0,1041.7,2503.8,11149.5,11829.6,3940.3,4347.3,2442.4,3549.7,845.3,2835.8,13953.4,8163.1,2479.8,10378.8,4811.6],
[3829.7,484.3,860.4,2216.3,1314.3,336.4,890.8,5489.4,2268.7,2361.1,700.1,844.2,1066.9,676.7,821.8,2128.8,2782.2,1255.2,2786.7,4587.2,4491.2,1023.6,8035.1,1787.4,1778,903.1,1843.2,4513.4,1818.1,1063.3,7576.3,558.5,9069.3,12321.2,7785.9,7576.3,1050.7,7330,11528.9,1041.7,0,1830,10475.6,11155.8,3969.9,3673.5,1768.6,2875.9,274.9,2162,13983,9175,1806,9705,4841.2],
[2368.7,1426.5,1054.1,460,872.7,2104.7,943.8,6507.1,620.8,1461,1609.2,1546.3,2654.6,1201.3,1524,1228.7,1882,2520.7,1080.7,5604.9,5508.9,2356.3,6574,315.9,307.9,1605.2,13.3,5531.1,315.1,2328.8,8594,1140.7,10087,13338.9,6885.7,8594,2383.3,6429.9,10628.8,2374.3,1699,0,9575.5,10255.7,4987.6,2773.3,298.4,1169.9,1862.7,1261.9,15000.7,10440.5,294,8804.9,5858.9],
[11877.7,10551.3,10178.9,9421.6,9229.4,11229.5,10068.6,16016.1,8992.2,8114.8,10734,10671.1,11779.4,10326,10648.8,8376.8,7756.3,11645.5,9510.1,15113.9,15018,11481,16083.1,9205,9213.5,10730,9229.3,15040.1,9204.2,11453.6,18103,10265.4,19596,22848,7547.9,18103,11508.1,3878.6,4673.1,11499.1,10823.8,9216,0,4300,14496.6,10481.3,9222.5,9599.4,10987.5,8540.4,24509.7,19565.3,9192,903.6,15367.9],
[12557.9,11231.4,10859,10101.8,9909.6,11909.6,10748.7,16696.3,9672.3,8795,11414.1,11351.3,12459.6,11006.2,11328.9,9056.9,8436.5,12325.6,10190.3,15794.1,15698.1,12161.2,11516.9,9885.2,9893.6,11410.2,9909.5,15720.3,9884.3,12133.8,18783.2,10945.6,20276.2,23528.2,8228.1,18783.2,12188.3,4558.8,2781.4,12179.2,11504,9896.2,4300,0,15176.8,11161.4,9902.7,10279.5,11667.7,9220.6,25189.9,20245.4,9872.2,3529.4,16048.1],
[3972.1,3820.4,3864.4,5189.8,4412.5,3761.5,4359.7,1543.2,5241.1,5333.6,3842.4,3553.2,3929.1,4145.7,3530.8,5101.3,5754.6,4117.4,5759.1,641,768.4,3885.9,8177.4,5155.6,5146.2,3612.1,5073.4,790.5,5048.2,3925.5,3630.1,4027.4,5123.1,8375.1,9775.3,3630.1,3913,10302.5,13390.8,3903.9,3933.5,5060.1,13448.1,14128.3,0,6645.9,5136.8,6606.6,3737.1,5134.5,10036.8,8477.8,5027.2,12677.5,895],
[4779.8,3453.3,3080.9,3413.7,1642.8,4131.5,2970.6,8918.2,2984.3,2366.8,3636,3573.2,4681.5,3228.1,3550.8,2368.9,2885,4547.5,3502.2,8016,7920,4383.1,8985.1,2737.5,2728,3632.1,3221.4,7942.2,3196.3,4355.6,11005.1,3167.5,12498.1,15750,7791.5,11005.1,4410.2,7335.7,11534.6,4401.1,3725.8,3208.2,10481.3,11161.4,7398.7,0,2718.6,3591.5,3889.5,2532.5,17411.8,13674.6,3184.1,9710.7,8270],
[2432.1,1489.9,1117.5,360.8,936.1,2168.1,1007.2,6570.5,453.1,1047.4,1672.6,1609.7,2718,1264.7,1587.4,815.1,1468.4,2584.1,981.5,5668.3,5572.3,2419.6,6637.4,19,9.5,1668.6,76.7,5594.5,51.5,2392.2,8657.4,1204.1,10150.4,13402.3,6472.1,8657.4,2446.7,6016.3,10215.2,2437.7,1762.4,63.4,9161.9,9842.1,5051,2836.7,0,1070.8,1926.1,848.3,15064.1,10503.9,39.4,8391.3,5922.3],
[3059.4,2853,2480.6,817.9,1963.3,3531.2,2370.3,7197.8,916.1,1539.5,3035.7,2972.8,4081.1,2627.8,2950.5,1307.2,1960.6,3947.2,643.7,6295.6,6199.6,3782.8,7264.7,1122.1,1130.5,3031.7,1146.4,6221.7,1121.2,3755.3,9284.7,2567.2,10777.7,14029.6,5927.5,9284.7,3809.8,6508.5,9543.1,3800.8,3125.5,1133.1,9654.1,10334.2,5678.2,3646.2,1139.6,0,3289.2,1340.4,15691.4,11867,1109.1,8883.4,6549.6],
[3993.4,648,1024.1,2380,1478,320.5,1054.5,5293,2432.4,2524.8,863.8,1007.9,870.4,840.4,985.5,2292.5,2945.9,1058.8,2950.4,4390.8,4294.8,827.2,8198.8,1951.1,1941.7,1066.8,2006.9,4317,1981.8,866.9,7379.9,722.2,8872.9,12124.8,7949.6,7379.9,854.3,7493.7,11692.6,845.3,274.9,1993.7,10639.3,11319.5,3773.5,3837.2,1932.3,3039.6,0,2325.7,13786.6,8978.5,1969.7,9868.7,4644.8],
[3019.6,1693.2,1320.8,1047.8,803.5,2371.4,1210.5,7158.1,618.4,710.8,1875.9,1813,2921.3,1468,1790.7,478.5,1131.9,2787.4,1136.4,6255.9,6159.9,2623,7225,831.2,839.7,1871.9,855.6,6182,830.4,2595.5,9244.9,1407.4,10737.9,13989.9,6135.6,9244.9,2650,5679.8,9878.6,2641,1965.7,842.3,8825.4,9505.5,5638.5,2817.5,848.7,1225.6,2129.4,0,15651.7,10707.2,818.3,8054.7,6509.9],
[11980.2,12460.9,12504.9,13830.3,13052.9,12401.9,13000.2,8039.7,13881.6,13974,12482.9,12193.6,12569.6,12786.1,12171.3,13741.7,14395.1,12757.9,14578.6,8023.2,9429.5,12526.4,13233.3,13796.1,13786.6,12252.6,13713.8,9451.6,13688.7,12566,11876.8,12667.9,13369.8,16621.8,17103.5,11876.8,12553.5,18943,20719,12544.4,12574,13700.6,22088.6,22768.7,8664.1,15286.4,13777.2,13934.8,12377.6,13774.9,0,16724.6,13667.6,21317.9,8100.4],
[12426.2,9224.4,9600.5,10956.3,10054.4,8664,9630.8,9603.6,11008.7,11101.2,9440.2,9584.2,8303.6,9416.7,9561.8,10868.9,11522.2,8034.2,11526.7,8701.4,9243.1,8172.9,16631.6,10527.5,10518,9643.1,10583.3,9265.3,10558.1,8111.7,8178.6,9298.5,3513.7,5396.2,16525.9,8178.6,8154,16070.1,20269,8163.1,9175,10570,19215.7,19895.8,8477.8,13674.6,10508.6,11615.9,8978.5,10902,18097.2,0,10546,18445.1,8955.4],
[2401.6,1459.4,1087,321.4,905.6,2137.6,976.7,6540,490.5,1084.8,1642.1,1579.2,2687.5,1234.2,1556.9,852.5,1505.8,2553.6,942.2,5637.8,5541.8,2389.2,6606.9,21.9,30.4,1638.1,46.2,5564,21,2361.7,8626.9,1173.6,10119.9,13371.8,6509.5,8626.9,2416.2,6053.7,10252.6,2407.2,1731.9,32.9,9199.3,9879.5,5020.5,2806.2,39.4,1031.4,1895.6,885.7,15033.6,10473.4,0,8428.7,5891.8],
[11107.1,9780.7,9408.2,8651,8458.8,10458.9,9297.9,15245.5,8221.5,7344.2,9963.4,9900.5,11008.8,9555.4,9878.1,7606.1,6985.7,10874.9,8739.5,14343.3,14247.4,10710.4,15312.5,8434.4,8442.9,9959.4,8458.7,14269.5,8433.5,10683,17332.4,9494.8,18825.4,22077.4,6777.3,17332.4,10737.5,3108,3902.5,10728.4,10053.2,8445.4,903.6,3529.4,13726,9710.7,8451.9,8828.7,10216.9,7769.8,23739.1,18794.7,8421.4,0,14597.3],
[4843.4,4691.8,4735.8,6061.1,5283.8,4632.8,5231,979.5,6112.5,6204.9,4713.8,4424.5,4800.5,5017,4402.2,5972.6,6626,4988.8,6630.4,254,1036.2,4757.2,6510.7,6026.9,6017.5,4483.4,5944.7,1058.4,5919.5,4796.9,4107.7,4898.7,5600.7,8852.7,10646.6,4107.7,4784.3,11173.8,14262.2,4775.3,4804.8,5931.4,14319.4,14999.6,895,7517.2,6008.1,7478,4608.4,6005.8,9473.1,8955.4,5898.5,13548.8,0]
       ]
    data['demands'] = [0, 0.64, 3.04, 1.41, 0.67, 4, 0.51, 0.94, 2, 3.2, 3.12, 4.79, 2.28, 1.25, 2.81, 4.12, 4.35, 2.66, 4.05, 3.39, 0.25, 0.88, 4.03, 0.55, 1.26, 2.07, 4.99, 0.93, 1.44, 1.65, 1.61, 1.19, 0.44, 4.49, 1.46, 3.63, 3.1, 1.01, 3.67, 4.81, 2.59, 4.06, 0.32, 2.52, 3.5, 3.18, 3.06, 3.62, 4.85, 3.84, 3.32, 3.63, 0.72, 0.53, 4.56]
    data['vehicle_capacities'] = [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
    data['num_vehicles'] = 15
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()

Objective: 193109
Route for vehicle 0:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 1:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 2:
 0 Load(0) ->  44 Load(3.5) ->  19 Load(6.890000000000001) ->  7 Load(7.83) ->  50 Load(11.15) ->  0 Load(11.15)
Distance of the route: 26907m
Load of the route: 11.15

Route for vehicle 3:
 0 Load(0) ->  27 Load(0.93) ->  20 Load(1.1800000000000002) ->  54 Load(5.74) ->  30 Load(7.3500000000000005) ->  33 Load(11.84) ->  0 Load(11.84)
Distance of the route: 27990m
Load of the route: 11.84

Route for vehicle 4:
 0 Load(0) ->  22 Load(4.03) ->  38 Load(7.7) ->  43 Load(10.22) ->  42 Load(10.540000000000001) ->  53 Load(11.07) ->  52 Load(11.790000000000001) ->  0 Load(11.790000000000001)
Distance of the route: 31654m
Load of the route: 11.790000000000001

Route for vehicle 5:
 0 Load(0) ->  21 Load(0.88) ->  36 Load(3.98) ->  39 Load(8.79) ->  12 Load(11.069999999